# Evaluación Práctica 3

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Clase de Simulación de Datos


La clase `SimuladorDatos` tiene como tarea generar un conjunto de datos simulados para
entrenar un modelo de Machine Learning que prediga si un estudiante abandonará o no sus
estudios.

In [ ]:
class SimuladorDatos:
    def __init__(self, n=200, seed=42):
        self.n = n
        self.seed = seed
        np.random.seed(self.seed)
    
    def generar(self):
        data = {
            'Edad': np.random.randint(18, 31, size=self.n),
            'Horas_estudio': np.random.uniform(0, 30, size=self.n).round(2),
            'Asistencia': np.random.uniform(50, 100, size=self.n).round(2),
            'Promedio': np.random.uniform(5, 10, size=self.n).round(2),
            'Uso_online': np.random.uniform(0, 15, size=self.n).round(2),
            'Abandono': np.random.choice([0, 1], size=self.n, p=[0.7, 0.3])
        }

        df = pd.DataFrame(data)
        
        return df


Pequeño ejemplo del uso de la clase

In [ ]:
simulador = SimuladorDatos(n=200, seed=42)
df = simulador.generar()
df.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import tree

## Clase para la Creación, Entrenamiento, Evaluación del Modelo


La clase `ModeloAbandono` se encargará de entrenar un modelo de Machine Learning para
predecir si un estudiante abandonará o no sus estudios, basándose en las características
proporcionadas en el conjunto de datos generado anteriormente (como edad, horas de estudio,
asistencia, etc.).

In [ ]:
class ModeloAbandono:
    def __init__(self, max_depth=4, random_state=42):
        self.max_depth = max_depth
        self.random_state = random_state
        self.modelo = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
    
    def entrenar(self, data):
        X = data.drop('Abandono', axis=1)
        y = data['Abandono']
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=self.random_state)

        self.modelo.fit(self.X_train, self.y_train)
        
        plt.figure(figsize=(16, 10))
        tree.plot_tree(self.modelo, filled=True, feature_names=X.columns, class_names=['Seguirá', 'Abandonará'])
    
    def evaluar(self):
        y_pred = self.modelo.predict(self.X_test)
        
        accuracy = accuracy_score(self.y_test, y_pred)
        print(f"Precisión del modelo: {accuracy:.2f}")
        print("\nReporte de clasificación:")
        print(classification_report(self.y_test, y_pred, target_names=['Seguirá', 'Abandonará']))
        
        return accuracy
    
    def predecir_estudiante(self, estudiante_df):
        prediccion = self.modelo.predict(estudiante_df)
        
        return "Abandonará" if prediccion[0] == 1 else "Seguirá estudiando"


## Clase para Testing del Modelo


La clase `TestBasicoModeloAbandono` tiene como propósito hacer pruebas simples para validar
que la solución de predicción del abandono estudiantil funciona correctamente.

In [ ]:
class TestBasicoModeloAbandono:
    def __init__(self, n_estudiantes=200, seed=42, max_depth=4):
        self.n_estudiantes = n_estudiantes
        self.seed = seed
        self.max_depth = max_depth
    
    def ejecutar(self):
        print("="*50)
        print("Iniciando prueba básica del modelo de abandono")
        print("="*50)
        
        print("\n[Paso 1/4] Generando datos simulados...")
        simulador = SimuladorDatos(n=self.n_estudiantes, seed=self.seed)
        datos_estudiantes = simulador.generar()
        print(f"✓ Se generaron {len(datos_estudiantes)} registros de estudiantes")
        print("   Muestra de los datos generados:")
        print(datos_estudiantes.head())
        
        print("\n[Paso 2/4] Entrenando el modelo...")
        modelo = ModeloAbandono(max_depth=self.max_depth, random_state=self.seed)
        modelo.entrenar(datos_estudiantes)
        print("✓ Modelo entrenado exitosamente")
        
        print("\n[Paso 3/4] Evaluando el modelo...")
        precision = modelo.evaluar()
        
        print("\n[Paso 4/4] Realizando predicción para nuevo estudiante...")
        nuevo_estudiante = self._crear_estudiante_ejemplo()
        print("\nDatos del nuevo estudiante:")
        print(nuevo_estudiante)
        
        resultado = modelo.predecir_estudiante(nuevo_estudiante)
        print(f"\nPredicción: El estudiante {resultado}")
        
        print("\n" + "="*50)
        print("Prueba completada exitosamente")
        print("="*50)
    
    def _crear_estudiante_ejemplo(self):
        crear_estudiante = SimuladorDatos(n=1, seed=69)
        df_estudiante = crear_estudiante.generar()
        return df_estudiante.drop('Abandono', axis=1)

## Preobando el Modelo Usando la Clase de Testing


Se asignan los parámetros correspondientes y se prueba el funcionamiendo de las clases

In [ ]:
test = TestBasicoModeloAbandono(n_estudiantes=200, seed=42, max_depth=3)
test.ejecutar()